# Aplicación web: LinkedIn
Grupo: Agarrini la Palini\
Integrantes: Adolfo Rojas, Benjamín Durán, Camila Paredes, Tomás Albornoz

## Descripción del Dataset y fuente
El dataset escogido es una colección de CSV files provenientes de API scraping de Linkedin con su listado de ofertas laborales. De momento se planea utilizar la versión publicada en [Kaggle](https://www.kaggle.com/datasets/arshkon/linkedin-job-postings/data) (9 meses desde la última actualización con >124,000 ofertas laborales enlistadas del 2023-2024)

### postings.csv

Contiene detalles de publicaciones de empleo en LinkedIn.

| Campo                    | Descripción                                                                 |
|--------------------------|-----------------------------------------------------------------------------|
| `job_id`                 | ID del empleo según LinkedIn ([enlace](https://www.linkedin.com/jobs/view/job_id)) |
| `company_id`            | Identificador de la empresa (relacionado con `companies.csv`)               |
| `title`                 | Título del trabajo                                                          |
| `description`           | Descripción del trabajo                                                     |
| `max_salary`            | Salario máximo                                                              |
| `med_salary`            | Salario medio                                                               |
| `min_salary`            | Salario mínimo                                                              |
| `pay_period`            | Período de pago (Hourly, Monthly, Yearly)                                   |
| `formatted_work_type`  | Tipo de trabajo (Fulltime, Parttime, Contract)                               |
| `location`              | Ubicación del trabajo                                                       |
| `applies`               | Número de postulaciones                                                     |
| `original_listed_time` | Tiempo original de publicación                                               |
| `remote_allowed`        | Si permite trabajo remoto                                                   |
| `views`                 | Número de visualizaciones                                                   |
| `job_posting_url`       | URL de la publicación                                                       |
| `application_url`       | URL para enviar postulaciones                                               |
| `application_type`      | Tipo de proceso de aplicación (offsite, complejo/simple onsite)             |
| `expiry`                | Fecha de expiración de la oferta                                            |
| `closed_time`           | Fecha de cierre de la publicación                                           |
| `formatted_experience_level` | Nivel de experiencia requerido (entry, associate, executive, etc)       |
| `skills_desc`           | Descripción de habilidades requeridas                                       |
| `listed_time`           | Fecha de publicación                                                        |
| `posting_domain`        | Dominio del sitio con el formulario de aplicación                           |
| `sponsored`             | Indica si la publicación está patrocinada                                   |
| `work_type`             | Tipo de trabajo asociado                                                    |
| `currency`              | Moneda del salario                                                          |
| `compensation_type`     | Tipo de compensación                                                        |

### company_details/companies.csv

Detalles de las empresas asociadas con las publicaciones de empleo.

| Campo         | Descripción                                                                 |
|---------------|-----------------------------------------------------------------------------|
| `company_id`  | ID de la empresa según LinkedIn                                             |
| `name`        | Nombre de la empresa                                                        |
| `description` | Descripción de la empresa                                                   |
| `company_size`| Tamaño de la empresa (0 más pequeña - 7 más grande)                         |
| `country`     | País de la sede                                                             |
| `state`       | Estado de la sede                                                           |
| `city`        | Ciudad de la sede                                                           |
| `zip_code`    | Código postal de la sede                                                    |
| `address`     | Dirección de la sede                                                        |
| `url`         | Enlace al perfil de LinkedIn de la empresa                                  |

### company_details/company_industries

| Columna      | Descripción                                               |
|---------------|-----------------------------------------------------------|
| `company_id` | ID de la empresa (PK, referencia a `companies`)           |
| `industry`   | ID de la industria                                        |

### company_details/company_specialities

| Columna      | Descripción                                               |
|---------------|-----------------------------------------------------------|
| `company_id` | ID de la empresa (PK, referencia a `companies`)           |
| `speciality` | ID de la especialidad                                     |


### company_details/employee_counts.csv

Registro del número de empleados y seguidores de las empresas.

| Campo            | Descripción                                         |
|------------------|-----------------------------------------------------|
| `company_id`     | ID de la empresa                                    |
| `employee_count` | Número de empleados en la empresa                   |
| `follower_count` | Número de seguidores en LinkedIn                    |
| `time_recorded`  | Timestamp en formato Unix de cuándo se recolectaron los datos |


### job_details/benefits.csv

Contiene información sobre los beneficios ofrecidos en los empleos.

| Campo     | Descripción                                                                  |
|-----------|------------------------------------------------------------------------------|
| `job_id`  | ID del empleo                                                                |
| `type`    | Tipo de beneficio (401K, Seguro Médico, etc)                                 |
| `inferred`| Indica si el beneficio fue etiquetado explícitamente o inferido por LinkedIn |

### job_details/job_industries

| Columna        | Descripción                                       |
|----------------|---------------------------------------------------|
| `job_id`       | ID del trabajo (PK, referencia a `postings`)          |
| `industry_id`  | ID de la industria (referencia a `industries`)    |


### job_details/job_skills

| Columna       | Descripción                                       |
|----------------|---------------------------------------------------|
| `job_id`      | ID del trabajo (PK, referencia a `postings`)          |
| `skill_abr`   | Clave de habilidad (referencia a `skills`)        |


### job_details/salaries

| Columna             | Descripción                                                     |
|---------------------|-----------------------------------------------------------------|
| `salary_id`         | ID del salario                                                  |
| `job_id`            | ID del trabajo (referencia a la tabla `postings`)               |
| `max_salary`        | Salario máximo                                                  |
| `med_salary`        | Salario medio                                                   |
| `min_salary`        | Salario mínimo                                                  |
| `pay_period`        | Periodo de pago (Por hora, Mensual, Anual)                      |
| `currency`          | Moneda del salario                                              |
| `compensation_type` | Tipo de compensación (Fija, Variable, etc.)                     |

## EDA

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

df_postings = pd.read_csv("postings.csv", encoding="UTF-8", engine="python")
df_companies = pd.read_csv("company_details/companies.csv", encoding="UTF-8", engine="python")

dfs = {
    "postings": df_postings,
    "companies": df_companies,
}

def admite_multiples(df, col):
    if df[col].dtype == object:
        return df[col].dropna().str.contains(r'[,\|;/]').any()
    return False

for nombre, dataframe in dfs.items():
    dimensiones = dataframe.shape
    nulos = dataframe.isnull().sum()
    unicos = dataframe.nunique()

    resumen = pd.DataFrame({
        "Valores únicos": unicos,
        "Valores nulos": nulos,
        "Porcentaje de nulos (%)": (nulos/dimensiones[0]*100).round(2),
        "Tipo de dato": dataframe.dtypes,
        "Admite múltiples": dataframe.columns.to_series().apply(lambda col: admite_multiples(dataframe, col))
    })
    
    display(Markdown(f"## {nombre}"))
    display(Markdown(f"Filas totales: {dimensiones[0]}"))
    display(Markdown(f"Columnas totales: {dimensiones[1]}"))
    display(resumen)

    num_registros = 5
    display(Markdown(f"### Primeros {num_registros} registros de {nombre}"))
    display(dataframe.head(num_registros))

## postings

Filas totales: 123849

Columnas totales: 31

,Valores únicos,Valores nulos,Porcentaje de nulos (%),Tipo de dato,Admite múltiples
job_id,123849,0,0.00,int64,False
company_name,24428,1719,1.39,object,True
title,72521,0,0.00,object,True
description,107827,7,0.01,object,True
max_salary,5321,94056,75.94,float64,False
pay_period,5,87776,70.87,object,False
location,8526,0,0.00,object,True
company_id,24474,1717,1.39,float64,False
views,684,1689,1.36,float64,False
med_salary,1417,117569,94.93,float64,False


### Primeros 5 registros de postings

...

Filas totales: 24473

Columnas totales: 10

,Valores únicos,Valores nulos,Porcentaje de nulos (%),Tipo de dato,Admite múltiples
company_id,24473,0,0.00,int64,False
name,24428,1,0.00,object,True
description,24164,297,1.21,object,True
company_size,7,2774,11.33,float64,False
state,788,22,0.09,object,True
country,81,0,0.00,object,False
city,4124,1,0.00,object,True
zip_code,7779,28,0.11,object,True
address,19476,22,0.09,object,True
url,24473,0,0.00,object,True


### Primeros 5 registros de companies

,company_id,name,description,company_size,state,country,city,zip_code,address,url
0,1009,IBM,"At IBM, we do more than work. We create. We cr...",7.0,NY,US,"Armonk, New York",10504,International Business Machines Corp.,https://www.linkedin.com/company/ibm
1,1016,GE HealthCare,Every day millions of people feel the impact o...,7.0,0,US,Chicago,0,-,https://www.linkedin.com/company/gehealthcare
2,1025,Hewlett Packard Enterprise,Official LinkedIn of Hewlett Packard Enterpris...,7.0,Texas,US,Houston,77389,1701 E Mossy Oaks Rd Spring,https://www.linkedin.com/company/hewlett-packa...
3,1028,Oracle,We’re a cloud technology company that provides...,7.0,Texas,US,Austin,78741,2300 Oracle Way,https://www.linkedin.com/company/oracle
4,1033,Accenture,Accenture is a leading global professional ser...,7.0,0,IE,Dublin 2,0,Grand Canal Harbour,https://www.linkedin.com/company/accenture


## Funcionalidades: 
- CRUD y filtraciones de ofertas laborales

## Consultas parametrizadas
1. ¿Qué empleos requieren la habilidad `[skill]`?\
    Join entre `job_postings` y `job_skills`

2. ¿Qué empleos fueron publicados por la empresa `[company_name]`?\
    Join entre `job_postings` y `companies`
     
4. ¿Qué empleos están disponibles en la ciudad `[city]` con modalidad `[work_type]`?\
    Filtro sobre `location` y `formatted_work_type`

6. ¿Qué empleos tienen un salario medio mayor a `[cantidad]` en la moneda `[currency]`?\
    Filtro sobre `med_salary` y `currency`

8. ¿Qué empleos permiten trabajo remoto y fueron publicados después de `[fecha]`?\
    Filtro sobre `remote_allowed` y `listed_time`